In [1]:
import pickle

filename = "ALL_LOGS.txt"


In [18]:
def calc_pr(players):
    pr = {}
    for player, (elo, score, games) in players.items():
        avg_opp = sum(e for p, (e, s, g) in players.items() if p != player) / (len(players) - 1)
        pr[player] = avg_opp + 400 * score / games
        
    return pr

In [19]:
def process_results(filename):
    players = {}  # Initialize an empty dictionary for players
    
    with open(filename, 'r') as file:
        for line in file:
            data = line.split()  # Split each line by spaces
            if len(data) < 3:
                continue  # Skip lines that don't have enough data
            
            i, j, outcome = data[0], data[1], int(data[2])
            
            # Ensure both players are in the dictionary
            if i not in players:
                players[i] = [0, 0]  # Score, Games
            if j not in players:
                players[j] = [0, 0]  # Score, Games
            
            # Update games played
            players[i][1] += 1
            players[j][1] += 1
            
            # Update scores based on the outcome
            if outcome == 0:  # Player i wins
                players[i][0] += 1
                players[j][0] -= 1
            elif outcome == 1:  # Player j wins
                players[j][0] += 1
                players[i][0] -= 1
            # No score change if it's a draw (-1)

    return players

In [20]:
players = process_results(filename)


In [21]:
def calc_elo(players, num_iter=10):
    '''
    Calculates relative elo such that all players have a elo equal to their performance rating

    players: dict of key: <player_name>, value: tuple (<score>, <games>) where score is (#wins - #losses) and games is number of games played
    i.e. a player that wins all games has score == games and a player that loses all games has score == -games
    requries that no player have won all or lost all games for convergence (otherwise elo = +- infinity)
    '''
    d = {player : (1000, score, games) for player, (score, games) in players.items()}
    for i in range(num_iter):
        pr = calc_pr(d)
        d = {player: (pr[player], score, games) for player, (elo, score, games) in d.items()}
        # d = {player:pr[player] for player, _ in d.items()}
    return d

In [22]:
elo_other = calc_elo(players, 10)

idx_to_name = {
    "0": "random",
    "1": "none_1_itrs",
    "2": "none_2_itrs",
    "3": "none_3_itrs",
    "4": "none_4_itrs",
    "5": "none_5_itrs",
    "6": "linear_1_itrs",
    "7": "linear_2_itrs",
    "8": "linear_3_itrs",
    "9": "linear_4_itrs",
    "10": "linear_5_itrs",
    "11": "pq_1_itrs",
    "12": "pq_2_itrs",
    "13": "pq_3_itrs",
    "14": "pq_4_itrs",
    "15": "pq_5_itrs",
    "16": "reset_1_itrs",
    "17": "reset_2_itrs",
    "18": "reset_3_itrs",
    "19": "reset_4_itrs",
    "20": "reset_5_itrs",
    "21": "sym_1_itrs",
    "22": "sym_2_itrs",
    "23": "sym_3_itrs",
    "24": "sym_4_itrs",
    "25": "sym_5_itrs",
    "26": "flamingo_1_itrs",
    "27": "flamingo_2_itrs",
    "28": "flamingo_3_itrs",
    "29": "flamingo_4_itrs",
    "30": "flamingo_5_itrs"
}

elo = {}
for key in elo_other:
    elo[idx_to_name[key]] = elo_other[key][0]

# write to pickle
with open('elos_2.pkl', 'wb') as f:
    pickle.dump(elo, f)